In [ ]:
!tar xvzf ../input/200-bird-species-with-11788-images/CUB_200_2011.tgz

In [ ]:
!head -70 CUB_200_2011/README

In [ ]:
import tracemalloc
tracemalloc.start()
import gc
import re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.data.experimental.enable_debug_mode()
import pandas as pd
import os
import cv2
import time
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

assert len(tf.config.experimental.list_physical_devices('GPU')) > 0, "No GPU found. Try to connect one..." #https://github.com/BaranovMykola/Tensorflow2Snippets/blob/master/image_dataset.ipynb

In [ ]:
ROOT = r"./CUB_200_2011"
IMAGES = ROOT + r"images/"
CHANNEL_NUM = 3
EXAMPLES_IN_CUB200 = 11788

IMAGE_SIZE = 224
NUM_CLASSES = 200
CLASSES_OFFSET = 0
BATCH_SIZE = 64

In [ ]:
ROOT = r"./CUB_200_2011"
IMAGES = ROOT + r"images/"
CHANNEL_NUM = 3
EXAMPLES_IN_CUB200 = 11788
#
IMAGE_SIDE = 224
NUM_CLASSES = 200
CLASSES_OFFSET = 0
BATCH_SIZE = 64

##################################################

def process_image(file_path): #https://github.com/BaranovMykola/Tensorflow2Snippets/blob/master/image_dataset.ipynb
    img = tf.io.read_file(tf.strings.join((ROOT, "images", file_path), separator=r"/"))
    img = tf.image.decode_jpeg(contents=img, channels=3)
    img = tf.image.resize(img, (IMAGE_SIDE,)*2)
    img /= 255.0
    return img

##################################################
#Read Files To pd.DataFrame
classes            = pd.read_csv(f"{ROOT}/classes.txt",            sep=" ", names=["class_id", "class_name"],        index_col="class_id")
classes            = list(classes["class_name"])
#
images             = pd.read_csv(f"{ROOT}/images.txt",             sep=" ", names=["image_id", "path"],              index_col="image_id")
train_test         = pd.read_csv(f"{ROOT}/train_test_split.txt",   sep=" ", names=["image_id", "is_training_image"], index_col="image_id").applymap(bool)
image_class_labels = pd.read_csv(f"{ROOT}/image_class_labels.txt", sep=" ", names=["image_id", "class_id"],          index_col="image_id")
##############################
def log(val):
    print(f"LOG : {val}")
    return val

train_test_ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(train_test.is_training_image.to_numpy()), #Get IS_TRAIN indicator
    tf.data.Dataset.from_tensor_slices(images.path.to_numpy()).map(process_image, num_parallel_calls=tf.data.AUTOTUNE), #Get paths and map them
    tf.data.Dataset.from_tensor_slices(image_class_labels.class_id.to_numpy()) #Get labels
)).filter(lambda is_train, image, label: label > CLASSES_OFFSET and label <= NUM_CLASSES + CLASSES_OFFSET) \
  .map(lambda is_train, image, label: (is_train, image, tf.one_hot(label-CLASSES_OFFSET-1, depth=NUM_CLASSES)), num_parallel_calls=tf.data.AUTOTUNE) #filter : get CLASS_NUM classes from some widow of with CLASSES_WINDOW_SELECTION_OFFSET; map : Subtract 1 and WINDOW_WIDTH to make indexes start with 0, and use One Hot

train = train_test_ds \
    .filter(lambda is_train, *_ : is_train) \
    .map(lambda *x: x[1:], num_parallel_calls=tf.data.AUTOTUNE) \
    .cache() \
    .shuffle(buffer_size=EXAMPLES_IN_CUB200)

test = train_test_ds \
    .filter(lambda is_train, *_ : not is_train) \
    .map(lambda *x: x[1:], num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH_SIZE, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
train = train \
    .batch(BATCH_SIZE, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
base = tf.keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224, 224, 3),
    )

base.trainable=False

# model 1: acc: 0.5395833253860474
# model = tf.keras.Sequential([
#     base,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=NUM_CLASSES, activation="sigmoid")
# ])

#model 2: 0.4833333194255829
# model = tf.keras.Sequential([
#     base,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=NUM_CLASSES, activation="sigmoid")
# ])

#model 3:0.5286458134651184
# model = tf.keras.Sequential([
#     base,
#     tf.keras.layers.GlobalMaxPooling2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=NUM_CLASSES, activation="sigmoid")
# ])

#model 4:0.4487847089767456
model = tf.keras.Sequential([
    base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=NUM_CLASSES, activation="sigmoid")
])

LEARNING_RATE = 0.0001
MOMENTUM = 0.99
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

In [ ]:
history=model.fit(train, epochs=50)

In [ ]:
plt.plot(history.history["accuracy"])

In [ ]:
model.evaluate(test)

In [ ]:
model.save('fully_connected.h5')